# Demasiado Ruido
Queremos simular la transmisión de texto a través de un canal ruidoso, donde algunos de los bits van a ser dañados. Veamos los detalles.

## Traducción
La información que se transmite consistirá en secuencias de bits, pero en origen es texto. Para convertir caracteres individuales a secuencias de bits y viceversa se proporciona el fichero `./files/charactcodif.txt` con encoding UTF-8. Un extracto:

```
...
_1100010
a1011100
b1000001
c0011001
d0111000
e1011110
f0000100
...
```

Cada línea empieza por el carácter a codificar, seguido de los 7 bits que lo representan.

Por ejemplo, la palabra `cada` se transformaría en la secuencia de 28 bits
`0011001101110001110001011100`.

## Canal
El canal es el medio por el que se transmite el mensaje, al ser ruidoso se caracteriza por su parámetro $\mu$, que es la probabilidad individual de que cada bit sea volteado (de modo que si era un 0, se convierte en 1 y viceversa). Todos los bits son susceptibles de ser volteados, independientemente unos de otros.

Por ejemplo, en el mensaje anterior, al simular la transmisión con un $\mu=0.04$ se han volteado tres bits:

<code>001100<span style="color: red;">0</span>101110001<span style="color: red;">0</span>10001<span style="color: red;">1</span>11100</code>.


Al recibir el mensaje en destino y traducirlo a texto resulta `5aN7`. Poco inteligible, ¿verdad?

## Códigos Redundantes
Dado que el canal destruye parte de la información, vamos a utilizar códigos que introducen redundancia para intentar combatirlo. Cada código viene caracterizado por una matriz, por ejemplo

$$
H_{47}=
\begin{pmatrix}
1 & 0 & 0 & 0 & 1 & 1 & 0 \\
0 & 1 & 0 & 0 & 1 & 0 & 1 \\
0 & 0 & 1 & 0 & 0 & 1 & 1 \\
0 & 0 & 0 & 1 & 1 & 1 & 1 \\
\end{pmatrix}
$$

Para codificar el mensaje añadiendo esa redundancia, se trocea en segmentos del tamaño del código (que es el número de filas de la matriz), en este caso 4:

`0011|0011|0111|0001|1100|0101|1100`.

Cada trozo se codifica, construyendo el vector fila con esos bits, multiplicándolo por la matriz y tomando el resto al dividir por 2:

Para el primer segmento `0011`:
$$
\begin{pmatrix}
0 & 0 & 1 & 1 \\
\end{pmatrix}
\cdot
\begin{pmatrix}
1 & 0 & 0 & 0 & 1 & 1 & 0 \\
0 & 1 & 0 & 0 & 1 & 0 & 1 \\
0 & 0 & 1 & 0 & 0 & 1 & 1 \\
0 & 0 & 0 & 1 & 1 & 1 & 1 \\
\end{pmatrix}
=
\begin{pmatrix}
0 & 0 & 1 & 1 & 1 & 2 & 2 \\
\end{pmatrix}
\to
\begin{pmatrix}
0 & 0 & 1 & 1 & 1 & 0 & 0 \\
\end{pmatrix}
$$

Con lo que se obtiene la secuencia de 7 bits `0011100`. Fíjate que los primeros dígitos son los que ya había, y a eso se añaden 3 más. Procesando los 7 trozos que teníamos, obtenemos:

`0011100|0011100|0111001|0001111|1100011|0101010|1100011`

que da lugar a la secuencia de 49 bits 

`0011100001110001110010001111110001101010101100011`

Este es el mensaje con redundancia incorporada que será transmitido por el canal.

## Recuperando la información
Una vez recibido el mensaje a través del canal (que habrá volteado algunos bits), necesitamos decodificarlo.

Para ello,
el mensaje recibido se trocea en segmentos de la longitud que produce el código, en este caso longitud 7.
Por ejemplo, supongamos que hubieramos recibido 
`1011100001110001110010001111110001101010101100011`. Los trozos son 

`1011100|0011100|0111001|0001111|1100011|0101010|1100011`

Fíjate que precisamente el primer bit se ha recibido como 1 aunque originalmente era un 0.

Tomamos el primer segmento `1011100`. Nos planteamos cuál debió ser el primer segmento de 4 bits del mensaje original, que ha quedado codificado con estos 7 bits. 
Dadas las $2^4 = 16$ posibles combinaciones de los 4 bits de partida, los resultados al codificar podrían ser:

```
0000000 0001111 0010011 0011100
0100101 0101010 0110110 0111001
1000110 1001001 1010101 1011010
1100011 1101100 1110000 1111111
```


El código encontrado `1011100` no coincide con ninguno de ellos, por lo que es seguro que algún bit fue volteado. Ahora bien, para que originariamente fuera `0000000` deberían haberse volteado 4 de los 7 bits, lo que es poco probable. El más probable de entre los 16 posibles es **el más *cercano*** a `1011100`, en el sentido de que el número de bits diferentes es menor. En particular se trata de `0011100`, que solo se diferencia en un bit. Por tanto, asumimos que ése fue el que realmente se transmitió. Por ser `0011100` el mensaje transmitido, el mensaje original sin redundancia lo formarían los primeros 4 bits: `0011`.

Se repite esta lógica con todos los segmentos de 7 bits transmitidos, y se van concatenando los prefijos de 4 bits asociados. Finalmente, cada bloque de 7 bits codifica un caracter según el fichero de texto proporcionado.

### Observación
Cuando antes decimos que se busca el más cercano, te podrías preguntar: ¿qué pasa si hay varios segmentos igual de cercanos? La matriz del código está hábilmente diseñada para que eso no suceda: siempre hay un código que es más cercano que cualquiera de los otros.

# Ejercicio 1: OOP
Queremos simular la transmisión de un mensaje con varios códigos diferentes, y distintos valores de $\mu$.

## Los códigos
Probaremos 3 códigos.

* El primero dado por la matriz

$$I_7=
\begin{pmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 \\
\end{pmatrix}
$$

Esta es la matriz identidad, que por tanto deja el mensaje original tal como estaba. Servirá para ver lo que pasa cuando transmitimos la información sin redundancia alguna.

* Un segundo código con inputs de 4 bits y outputs de 7 bits.
$$
H_{4,7}=
\begin{pmatrix}
1 & 0 & 0 & 0 & 1 & 1 & 0 \\
0 & 1 & 0 & 0 & 1 & 0 & 1 \\
0 & 0 & 1 & 0 & 0 & 1 & 1 \\
0 & 0 & 0 & 1 & 1 & 1 & 1 \\
\end{pmatrix}
$$

* Un último código con inputs de 12 bits y outputs de 23 bits.
$$
G_{12,23}=
\begin{pmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 1 & 1 & 1 & 1 & 1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 1 & 1 & 1 & 1 & 1 & 0 & 1 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 1 & 1 & 1 & 1 & 1 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 1 & 1 & 1 & 1 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 1 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 1 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 1 & 0 & 0 & 1 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 1 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 1 & 0 & 0 & 1 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 \\
\end{pmatrix}
$$



Para hacer la vida un poco más fácil, en Python:

In [1]:
import numpy as np
H_4_7 = np.array([
    [1, 0, 0, 0, 1, 1, 0],
    [0, 1, 0, 0, 1, 0, 1],
    [0, 0, 1, 0, 0, 1, 1],
    [0, 0, 0, 1, 1, 1, 1],
], dtype=np.uint8)

G_12_23 = np.array([
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
], dtype=np.uint8)

## El mensaje
Hemos elegido el comienzo del [The Zen of Python](https://peps.python.org/pep-0020/), en concreto:

In [2]:
message_text = "Beautiful is better than ugly. Explicit is better than implicit."

## Probabilidades de volteo
Vamos a tomar los valores $\mu=0.0025$, $\mu=0.01$, $\mu=0.03$ y $\mu=0.05$

## Objetivo
Se pide una implementación que pruebe a transmitir 5 veces el mensaje elegido con cada uno de los códigos, y valores de $\mu$. Se debe mostrar por pantalla el texto reinterpretado, así como el porcentaje de caracteres en los que ha cometido un error.

Es decir, se espera algo del estilo:

```
Simulations with probFlip=0.0025
I7 | 'Beautiful is better thaU ugly. ExplicNt is better than implicit.' | Errors: 3.12%
I7 | 'Beautiful is better than ugly. Explicit is better than implicit.' | Errors: 0.00%
I7 | 'Beautiful is better than ugly. Explicit is better t;an implicitB' | Errors: 3.12%
I7 | 'Beoutiful is better than ugly. Explicit is better than implicit.' | Errors: 1.56%
I7 | 'Beautiful is better than Tgly. Explicit is be?ter than implicit.' | Errors: 3.12%
H4_7 | 'Beautiful is better than ugly. Explicit is better than implicit.' | Errors: 0.00%
H4_7 | 'Beautiful is better than ugly. Explicit is better than implicit.' | Errors: 0.00%
H4_7 | 'Beautiful is better than ugly. Explicit is better than implicit.' | Errors: 0.00%
H4_7 | 'Beautiful is better than ugly. Explicit is better than implicit.' | Errors: 0.00%
H4_7 | 'Beautiful is better than ugly. Explicit is better than implicit.' | Errors: 0.00%
G12_23 | 'Beautiful is better than ugly. Explicit is better than implicit.' | Errors: 0.00%
G12_23 | 'Beautiful is better than ugly. Explicit is better than implicit.' | Errors: 0.00%
G12_23 | 'Beautiful is better than ugly. Explicit is better than implicit.' | Errors: 0.00%
G12_23 | 'Beautiful is better than ugly. Explicit is better than implicit.' | Errors: 0.00%
G12_23 | 'Beautiful is better than ugly. Explicit is better than implicit.' | Errors: 0.00%
Simulations with probFlip=0.0100
I7 | 'Beautiful is better than ugsy. Explic?t is better than implicit.' | Errors: 3.12%
I7 | 'Beaut/ful i% be,ter than ugly. Explicit is better than0implLcit.' | Errors: 7.81%
...
```

## Indicaciones
El ejercicio debe enfocarse orientado a objetos. Aunque la elección de clases y métodos es parte de lo que se evalúa, se sugieren:
* una clase `Translator` para gestionar la traducción entre texto y secuencias de bits dictada por el fichero proporcionado.
* una clase sencilla `Channel` podría simular la transmisión de mensajes alterando las secuencias de bits según $\mu$.
* una clase `RedundantCode` que reciba una matriz de código, con funcionalidades para la gestión de la traducción de mensajes, introducción de redundancia en el mensaje original, eliminación de redundancia e interpretación de mensajes recibidos, etc.

Para este ejercicio facilita bastante la vida tratar las secuencias de bits como `np.array` de dimensión y tipo `dtype=np.uint8`. Como los del código proporcionado para las matrices.


## Observaciones
* No todas las secuencias de 7 bits corresponden a un caracter. Podría pasarte que al reinterpretar un mensaje, finalmente busques el caracter para 7 bits concretos y en el fichero no exista. En ese caso, pon un carácter de interrogación `?`.

* ¿Qué hacer si tu mensaje tiene 27 bits y estás usando un código que necesita trozos de 12 bits? Puedes rellenar con 0s por la derecha, hasta tener un múltiplo de 12, en este caso completarías hasta 36 bits y tratarías los 3 trozos. Para luego saber que el original tenía 27 bits, esa longitud puedes pasarla junto con el mensaje codificado. 

# Ejercicio 2: no OOP
Resulta que para la matriz $G_{12,23}$, cuando partiendo de un segmento de 12 bits lo codificamos a  23 bits, se va a poder recuperar el mensaje original si de esos 23 bits se voltean exactamente 3 o menos. Eso nos permite calcular la probabilidad de que el bloque de 12 bits se reinterprete incorrectamente como:

$$p(\mu) = 1 - \sum_{k=0}^3 {23 \choose {k}} \mu^k (1-\mu)^{23-k}$$

Implementa esta función en Python y haz un plot para el intervalo $\mu\in[0, 0.05]$.

## Indicación
La notación $n \choose m$ representa los combinaciones de $n$ elementos tomados de $m$ en $m$, y puede calcularse mediante
$${n \choose m} = \frac{n!}{m! (n-m)!},$$
donde $k! = 1 \cdot 2 \cdot 3 \cdot \dots \cdot (k-1) \cdot k$ es el factorial de $k$.